    Problem Statement.

    There are n servers numbered from 0 to n-1 connected by undirected server-to-server connections forming a network where connections[i] = [a, b] represents a connection between servers a and b. Any server can reach any other server directly or indirectly through the network.

    A critical connection is a connection that, if removed, will make some server unable to reach some other server.

    Return all critical connections in the network in any order.



    Example 1:

    Input: n = 4, connections = [[0,1],[1,2],[2,0],[1,3]]
    Output: [[1,3]]
    Explanation: [[3,1]] is also accepted.



    Constraints:

        1 <= n <= 10^5
        n-1 <= connections.length <= 10^5
        connections[i][0] != connections[i][1]
        There are no repeated connections.

# BFS Time Limit Exceeded - O(N ^ 2) runtime, O(N) space

In [1]:
from typing import Dict, List, Set
from collections import deque

class Solution:
    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        
        graph = {g:set() for g in range(n)}
        
        for c in connections:
            graph[c[0]].add(c[1])
            graph[c[1]].add(c[0])
        
        res = []
        seen = set()
        for g in graph:
            if len(graph[g]) == 1:
                res.append([g, list(graph[g])[0]])
                seen.add((g, list(graph[g])[0]))
        
        for c in connections:
            if tuple(c) in seen or (c[1], c[0]) in seen:
                continue
                
            graph[c[0]].remove(c[1])
            graph[c[1]].remove(c[0])
            
            if not self.isConnected(graph, n):
                res.append(c)
            
            graph[c[0]].add(c[1])
            graph[c[1]].add(c[0])
                
        return res
        
    def isConnected(self, graph: Dict[int, Set[int]], n: int) -> bool:
        visited = {0}
        queue = deque(list(graph[0]))
        while queue:
            node = queue.popleft()
            visited.add(node)
            for c in graph[node]:
                if c not in visited:
                    queue.append(c)
                    
        return True if len(visited) == n else False

# DFS using depths - O(E) runtime, O(E) space

In [2]:
from typing import Dict, List, Set

class Solution:
    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        
        # graph construction
        graph = [[] for i in range(n)]
        for u, v in connections:
            graph[u].append(v)
            graph[v].append(u)
        
        # depth initialization
        depths = [-1] * n
        results = []
        
        """
        visit every node exactly once, the starting point does not matter
        (as long as graph is connected)
        """
        def dfs(prev_node, cur_node, cur_depth):
            depths[cur_node] = cur_depth
            min_depth = cur_depth
            for neighbor in graph[cur_node]:
                if neighbor == prev_node: continue
                """
                find the temporary depth reached by a neighbor
                """
                temp_depth = depths[neighbor]
                """
                if the node is unexplored,  assign it's depth to current depth + 1
                """
                if temp_depth == -1:
                    temp_depth = dfs(cur_node, neighbor, cur_depth+1)
                """
                if the returned depth is deeper than the "current depth", then it is a critical connection
                else, update the min_depth
                NOTE: we are comparing the "returned depth from neighbor (temp_dpeth)" to the "current depth reached by DFS" rather than the "min_depth" that is going to be returned.
                because once the temp_depth is returned by a neighbor, it is the minimum depth of it. 
                """
                if temp_depth > cur_depth:
                    results.append([cur_node, neighbor])
                else:
                    min_depth = min(min_depth, temp_depth)
            """
            return the minimum depth reached by a node
            """
            depths[cur_node] = min_depth
            return min_depth
        
        # start at node-0
        dfs(None, 0, 0)
        
        return results

In [3]:
instance = Solution()
instance.criticalConnections(4, [[0,1],[1,2],[2,0],[1,3]])

[[1, 3]]